# Proyecto de predicción de fugas en compañía telefónica - explotación

In [1]:
import pickle

In [2]:
with open("modelo.pkl", "rb") as f:
    opciones = pickle.load(f)

In [3]:
opciones

{'modelo': GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=26,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=None, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 'variables_transformadas_a_log': array(['Customer tenure in month',
        'Total Spend in Months 1 and 2 of 2017', 'Total SMS Spend',
        'Total Data Spend', 'Total Data Consumption', 'Total Unique Calls',
        'Total Onnet spend', 'Total Offnet spend',
        'Total Call

In [4]:
model                         = opciones["modelo"]
variables_transformadas_a_log = opciones["variables_transformadas_a_log"]
estandarizador                = opciones["estandarizador"]
variables_estandarizadas      = opciones["variables_estandarizadas"]
variables_usadas              = opciones["variables_usadas"]

In [5]:
import pandas as pd
import numpy as np

In [6]:
df_expl = pd.read_csv("../datasets/fuga_clientes_empresa_telefonica_explotacion.csv")
print(df_expl.shape)
aux2 = df_expl.isna().sum()
aux2[aux2>0]

(289, 16)


Network type subscription in Month 1    175
Network type subscription in Month 2    122
Churn Status                            289
dtype: int64

En vista de los resultados, se eliminan todas las filas de training que tienen algún nan en alguna columna. Por otra parte, se eliminan, tanto de training como de test, las variables **Network type subscription in Month 1** y **Network type subscription in Month 2**.

In [7]:
df_expl = df_expl.drop(["Network type subscription in Month 1",
                        "Network type subscription in Month 2"], axis=1)

In [8]:
df_expl.shape

(289, 14)

In [9]:
aux2 = df_expl.isna().sum()
aux2[aux2>0]

Churn Status    289
dtype: int64

In [10]:
df_expl_num = pd.get_dummies(df_expl.drop('Customer ID', axis=1))
df_expl_num.insert(loc=0, value=df_expl['Customer ID'], column='Customer ID')

In [11]:
# Por si posteriormente queremos trabajar directamente a partir de
# las bases de datos limpias:
df_expl_num.to_csv("fuga_clientes_empresa_telefonica_explotacion_LIMPIO.csv", index=None)

## Siguiente paso: trabajamos con la base de datos completamente numérica

In [12]:
df_expl = df_expl_num
del df_expl_num

In [13]:
df_expl[df_expl["network_age"]<0]

,Customer ID,network_age,Customer tenure in month,Total Spend in Months 1 and 2 of 2017,Total SMS Spend,Total Data Spend,Total Data Consumption,Total Unique Calls,Total Onnet spend,Total Offnet spend,...,Most Loved Competitor network in in Month 1_ToCall,Most Loved Competitor network in in Month 1_Uxaa,Most Loved Competitor network in in Month 1_Weematel,Most Loved Competitor network in in Month 1_Zintel,Most Loved Competitor network in in Month 2_Mango,Most Loved Competitor network in in Month 2_PQza,Most Loved Competitor network in in Month 2_ToCall,Most Loved Competitor network in in Month 2_Uxaa,Most Loved Competitor network in in Month 2_Weematel,Most Loved Competitor network in in Month 2_Zintel
52,ADF1623,-2,-0.07,130.578,94.85,1.25,2.1953,7,150,1121,...,0,1,0,0,0,0,0,1,0,0


In [14]:
df_expl = df_expl[df_expl["network_age"]>=0]

In [15]:
df_expl.describe().loc[["min","max"]]

,network_age,Customer tenure in month,Total Spend in Months 1 and 2 of 2017,Total SMS Spend,Total Data Spend,Total Data Consumption,Total Unique Calls,Total Onnet spend,Total Offnet spend,Total Call centre complaint calls,...,Most Loved Competitor network in in Month 1_ToCall,Most Loved Competitor network in in Month 1_Uxaa,Most Loved Competitor network in in Month 1_Weematel,Most Loved Competitor network in in Month 1_Zintel,Most Loved Competitor network in in Month 2_Mango,Most Loved Competitor network in in Month 2_PQza,Most Loved Competitor network in in Month 2_ToCall,Most Loved Competitor network in in Month 2_Uxaa,Most Loved Competitor network in in Month 2_Weematel,Most Loved Competitor network in in Month 2_Zintel
min,95.0,3.17,16.1520,0.00,0.00,1.172000e-01,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,5363.0,178.77,10042.3532,873.98,493.75,3.714204e+07,2596.0,381174.0,431440.0,16.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
df_expl.shape

(288, 25)

### Creación de variables sintéticas nuevas

In [17]:
df_expl["ratio2"] = df_expl["Total SMS Spend"] / (1 + df_expl["Total Offnet spend"] + df_expl["Total Onnet spend"])

df_expl["ratio3"] = df_expl["Total SMS Spend"] / df_expl["Total Unique Calls"]

df_expl["ratio4"] = df_expl["Total Spend in Months 1 and 2 of 2017"] * df_expl["Total Unique Calls"]

In [18]:
for name in variables_transformadas_a_log:
    df_expl[name] = np.log(1+df_expl[name])

### Estandarización de los datos

In [19]:
if estandarizador is not None:
    aux = estandarizador.transform(df_expl[variables_estandarizadas].values)
    df_expl[variables_estandarizadas] = aux

### Finalmente utilizo solo las variables usadas en el notebook de construcción del modelo

In [20]:
IDs_expl = df_expl["Customer ID"].values
df_expl = df_expl[variables_usadas]
X_expl = df_expl.values

In [21]:
variables_usadas

array(['Customer tenure in month',
       'Total Spend in Months 1 and 2 of 2017', 'Total SMS Spend',
       'Total Data Consumption', 'Total Unique Calls',
       'Total Onnet spend', 'Total Call centre complaint calls',
       'Most Loved Competitor network in in Month 1_0',
       'Most Loved Competitor network in in Month 1_Mango',
       'Most Loved Competitor network in in Month 1_PQza',
       'Most Loved Competitor network in in Month 1_Uxaa',
       'Most Loved Competitor network in in Month 1_Zintel',
       'Most Loved Competitor network in in Month 2_Mango',
       'Most Loved Competitor network in in Month 2_PQza',
       'Most Loved Competitor network in in Month 2_ToCall',
       'Most Loved Competitor network in in Month 2_Uxaa',
       'Most Loved Competitor network in in Month 2_Weematel',
       'Most Loved Competitor network in in Month 2_Zintel', 'ratio3',
       'ratio4'], dtype=object)

# Predicciones en explotación

In [22]:
prob_fuga_expl = model.predict_proba(X_expl)[:,1]

In [23]:
prob_fuga_expl[:5]

array([0.16603645, 0.44543021, 0.23944853, 0.38128976, 0.16560594])

In [24]:
predicciones = pd.DataFrame({"Customer ID":IDs_expl,
                             "Prob Fuga":prob_fuga_expl})
predicciones[:5]

,Customer ID,Prob Fuga
0,ADF1330,0.166036
1,ADF1331,0.445430
2,ADF1345,0.239449
3,ADF1349,0.381290
4,ADF1363,0.165606


In [25]:
predicciones = predicciones.sort_values("Prob Fuga",
                                        ascending=False)[:200]
predicciones.iloc[:5]

,Customer ID,Prob Fuga
140,ADF0391,0.919275
147,ADF0442,0.919275
141,ADF0409,0.918054
177,ADF0601,0.917867
101,ADF1896,0.913433


In [26]:
predicciones[["Customer ID"]].to_csv("top_100_v2.csv", index=None)